# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'human_nontata_promoters'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "demo_mouse_enhancers"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

demo_mouse_enhancers 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf
import numpy as np

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded('human_nontata_promoters'):
    download_dataset('human_nontata_promoters')

2021-10-23 00:13:57.743279: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-23 00:13:57.743293: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2021-10-23 00:14:00.552973: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-23 00:14:00.553018: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michle): /proc/driver/nvidia/version does not exist
2021-10-23 00:14:00.553526: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
info(DATASET)

Dataset `demo_mouse_enhancers` has 2 classes: negative, positive.

The lenght of genomic intervals ranges from 331 to 4776, with average 2369.5768595041322 and median 2381.0.

Totally 1210 sequences have been found, 968 for training and 242 for testing.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:96: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


,train,test
negative,484,121
positive,484,121


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 968 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-10-23 00:14:00.991052: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 'a', 'c', 't', 'g', 'n']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


 1/16 [>.............................] - ETA: 15s - loss: 0.9479 - binary_accuracy: 0.3281

 2/16 [==>...........................] - ETA: 3s - loss: 0.9111 - binary_accuracy: 0.2969 

 3/16 [====>.........................] - ETA: 3s - loss: 0.8575 - binary_accuracy: 0.3021

 4/16 [======>.......................] - ETA: 3s - loss: 0.8208 - binary_accuracy: 0.3320

...

13/16 [=======================>......] - ETA: 1s - loss: 0.5309 - binary_accuracy: 0.7404

14/16 [=========================>....] - ETA: 0s - loss: 0.5325 - binary_accuracy: 0.7411

15/16 [===========================>..] - ETA: 0s - loss: 0.5345 - binary_accuracy: 0.7406

16/16 [==============================] - ETA: 0s - loss: 0.5344 - binary_accuracy: 0.7407

16/16 [==============================] - 5s 333ms/step - loss: 0.5344 - binary_accuracy: 0.7407


## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 242 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

1/4 [======>.......................] - ETA: 0s - loss: 0.6297 - binary_accuracy: 0.6250

2/4 [==============>...............] - ETA: 0s - loss: 0.6045 - binary_accuracy: 0.6484

3/4 [=====================>........] - ETA: 0s - loss: 0.6414 - binary_accuracy: 0.5938

4/4 [==============================] - 0s 51ms/step - loss: 0.6554 - binary_accuracy: 0.5702


[0.6554393768310547, 0.5702479481697083]